<a href="https://colab.research.google.com/github/Cmput-414/PoinTr/blob/master/Project%20Implementation2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Step 1: Set environments**



In [ ]:
##########################################################Step 1: Git clone code ##################################################
# 1. we clone the code from the github.
# 2. This will create a directory /content/pointr
!git -C /content/ clone https://github.com/Cmput-414/PoinTr.git pointr
########################################################################################################################################

##########################################################Step 2: Install conda ####################################################
# 1. Install conda environment for later use.
# 2. We will use conda to install a vitural environments with python 3.6
!wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local/
!conda create --no-default-packages -n myenv python=3.6 --yes
########################################################################################################################################

##########################################################Step 3: set virtual environment #########################################
# 1. After we set virtual environment， we activate it, check if we are in correct environment.
# 2. Then we install correct cuda version and pytorch version.
# 3. Then we will start to install some other packages. (This step will take some times, around 10 minutes.
!source activate myenv && which python && conda install pytorch==1.4.0 torchvision==0.5.0 cudatoolkit=10.0 -c pytorch -y
!source activate myenv && pip install ninja
!source activate myenv && git config --global url."https://".insteadOf git://
!source activate myenv && pip install "git+git://github.com/erikwijmans/Pointnet2_PyTorch.git#egg=pointnet2_ops&subdirectory=pointnet2_ops_lib"
!source activate myenv && pip install --upgrade https://github.com/unlimblue/KNN_CUDA/releases/download/0.1/KNN_CUDA-0.1-py3-none-any.whl
%cd /content/pointr
!source activate myenv && pip install -r requirements.txt
!source activate myenv && cd /content/pointr/extensions/chamfer_dist && python setup.py install --user
!source activate myenv && cd /content/pointr/extensions/cubic_feature_sampling && python setup.py install --user
!source activate myenv && cd /content/pointr/extensions/gridding && python setup.py install --user 
!source activate myenv && cd /content/pointr/extensions/gridding_loss && python setup.py install --user
########################################################################################################################################

### **Step 2: Set dataset**

In [ ]:
# Please check your email, we send a shared google drive that contain the dataset.
# If you will not see that shared drive, you can download teh dataset from README.md / new_data.md
##################################################Step 1: Connect shared drive ####################################################
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# After set environment, we will start set dataset
# In this project, we will use KITTI and PCN dataset
##################################################Step 1: KITTI dataset ############################################################
#https://drive.google.com/file/d/1Vvci0uu-eAKsBrDEwJWWBP_r9qr_eHzI/view?usp=share_link

!cp -r '/content/drive/Shareddrives/Pointr/kitti/bboxes' /content/pointr/data/KITTI
!cp -r '/content/drive/Shareddrives/Pointr/kitti/cars' /content/pointr/data/KITTI
!cp -r '/content/drive/Shareddrives/Pointr/kitti/tracklets' /content/pointr/data/KITTI
########################################################################################################################################

##################################################Step 2: PCN dataset ##############################################################
# https://drive.google.com/file/d/1-L2bqK2dRMjSkDR8K6LEe_ad6JFT3kwj/view?usp=share_link

!unzip /content/drive/Shareddrives/Pointr/PCN.zip -d / 
########################################################################################################################################

### **Step 3: Train code**
In this step: we will get some check points, then we can use these check points to do test. Finally, we can get visual result from the training result.

In [ ]:
%cd /content/pointr
# You can change KITTI_models to PCN_models to train PCN dataset.
# The check point will save in to new direcory: ./experiments
# For example: save ckpt for KITTI
# /content/pointr/experiments/PoinTr/KITTI_models/example
# For example: save ckpt for PCN
# /content/pointr/experiments/PoinTr/PCN_models/example
# Our training based on PoinTr.yaml.
# PoinTr.yaml contain the information about total batch size(total_bs) and epoch. 
# You can change the batch size and epoch number before start training.
# For KITTI dataset, The PoinTr.yaml located in /content/pointr/cfgs/KITTI_models/PoinTr.yaml
# For PCN dataset, The PoinTr.yaml located in /content/pointr/cfgs/PCN_models/PoinTr.yaml
# You can move the mouse pointer to the above link, click it will direct you to that position. (Or press ctrl + click)

!source activate myenv && which python && bash ./scripts/train.sh 0 --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example

### **Step 4: Test code**
In this step, we use training result to do test, and get some visual result.
Alternatively,we provide some pretrain model to do evaluation.

In [ ]:
%cd /content/pointr
# You can choose one of check point to test.
# Our test based on PoinTr.yaml and check point from training.
# PoinTr.yaml contain the information about total batch size(total_bs) and epoch. 
# You can change the batch size and epoch number before start testing.
# Test and training should keep same batch size and epoch number.
# Find ckpt on ./experiements folder
# For example: load ckpt for KITTI
# /content/pointr/experiments/PoinTr/KITTI_models/example/ckpt-best.pth
# For example: load ckpt for PCN
# /content/pointr/experiments/PoinTr/PCN_models/example/ckpt-best.pth
!source activate myenv && which python && bash ./scripts/test.sh 0 --ckpts ./experiments/PoinTr/KITTI_models/example/ckpt-best.pth --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example

In [ ]:
%cd /content/pointr
# You can choose one of pretrained model to test.
# PoinTr.yaml contain the information about total batch size(total_bs) and epoch. 
# You cannot change the batch size and epoch number before start testing.
# Test should keep same batch size and epoch number with pretrained model.
# Download pretrained model on ./pretrained folder
!unzip /content/drive/Shareddrives/Pointr/pretrain/kitti-ckpt_best.pth -d /content/pointr/pretrained
!unzip /content/drive/Shareddrives/Pointr/pretrain/pcn-ckpt_best.pth -d /content/pointr/pretrained
# For example: load pretrained model for KITTI
# /content/pointr/pretrained/kitti-ckpt-best.pth
# For example: load pretrained model for PCN
# /content/pointr/pretrained/pcn-ckpt-best.pth
!source activate myenv && which python && bash ./scripts/test.sh 0 --ckpts ./pretrained/kitti-ckpt-best.pth --config ./cfgs/KITTI_models/PoinTr.yaml --exp_name example

# Extra step: See visual result

In [ ]:
# Run below code to see a gif image for the visual result.
# If you test PCN dataset, you should change KITTI_models to PCN_models
# The gif image will save into /content/pointr/visual_result.gif
# You can move the mouse pointer to the above link, click it will direct you to that position. (Or press ctrl + click)

import glob
from PIL import Image
fp_in = "/content/pointr/experiments/PoinTr/KITTI_models/test_example/vis_result/*.png"
fp_out="/content/pointr/visual_result.gif"
# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
imgs = (Image.open(f) for f in glob.glob(fp_in))
img = next(imgs)  # extract first image from iterator
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0)

In [ ]:
# Run below code to see video for the visual images.
# The code combine all images to a video. 
# The video will save into /content/pointr/video.avi
# You can move the mouse pointer to the above link, click it will direct you to that position. (Or press ctrl + click)
import cv2
import os

image_folder = '/content/pointr/experiments/PoinTr/KITTI_models/test_example/vis_result/'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 1, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()